In [1]:
import os
import shutil
#import genomepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""Build gene regulatory network"""
import math
import os
import re
import shutil
import sys
import warnings
from tempfile import NamedTemporaryFile, mkdtemp
from typing import Union

#import dask.dataframe as dd
#import genomepy
import numpy as np
import pandas as pd
#import pyranges as pr
#from loguru import logger
from scipy.stats import rankdata
from sklearn.preprocessing import minmax_scale
from tqdm.auto import tqdm

#from ananse import PACKAGE_DIR, SEPARATOR
#from ananse.utils import cleanpath
#from ananse.view import get_binding_tfs

warnings.filterwarnings("ignore")

## Try to handle RNA data properly

## Parse M2F's original orthofinder

In [ ]:
# run orthofinder
# define the load_orthogroups_in_db function
# run the load_orthogroups_in_db function to create the .sqlite database
# run by hand the makemotif2factors; maybe this is the optimal place where to place the checkings in rosetta etc??
# 


In [ ]:
# load orthofinder into data base
orthogroup_db = f"{tmpdir}/orthologs.sqlite"
load_orthogroups_in_db(orthogroup_db, all_genomes, orthofinder_result)

In [ ]:
# with orthofinder loaded, load more evidences

In [ ]:
# check if we can add here the motif database that we do on our own??

In [ ]:
# I NEED TO BRING HERE THE motifsandfactors THINGY

In [ ]:
# def make_motif2factors(
#    prefix,
#    new_reference,
#    database_references,
#    motifsandfactors,
#    database,
#    strategy,
#    motifs,
# ):
#     """
#     Make a motifs2factors file based on an existing ortholog database.

#     We loop over each motif, find which factors belong to it in the old
#     reference, then find all the orthogroups those belong to, and finally assign
#     all the genes of the new reference species that belong to those orthogroups.
#     """
    factor2orthogroups.cache_clear()
    conn = sqlite3.connect(database)
    cur = conn.cursor()

    with open(f"{prefix}.motif2factors.txt", "w") as f: # turn this into a dataframe that gets filled. We will write the final table later
        f.write("Motif\tFactor\tEvidence\tCurated\n")
        for motif, factors in motifsandfactors.items():
            # remember if we found any orthologs for all the factors belonging
            # to our motif
            motif_set = False

            # get all the orthogroups belonging to all our factors belonging to
            # our motif. This is not as trivial as it sounds, since gene naming
            # is a mess
            # <-- in relation to this last comment, we need to dig if Jakke gave these TFs a human name
            orthologousgroups = {
                item
                for factor in factors
                for item in factor2orthogroups(
                    factor, tuple(database_references), database, strategy
                )
            }
            if len(orthologousgroups) > 0:
                for orthologousgroup in orthologousgroups:
                    res = cur.execute(
                        f"SELECT * FROM genes WHERE orthogroup='{orthologousgroup}' AND assembly='{new_reference}'"
                    ).fetchall()

                    if len(res):
                        for _, gene_name, _, gene_id, *_ in res:
                            motif_set = True
                            if gene_name is not None:
                                f.write(f"{motif}\t{gene_name}\tOrthologs\tN\n")
                            else:
                                f.write(f"{motif}\t{gene_id}\tOrthologs\tN\n")
            if not motif_set:
                f.write(f"{motif}\tNO ORTHOLOGS FOUND\tOrthologs\tN\n")

    with open(f"{prefix}.pfm", "w") as f:
        for motif in motifs:
            print(motif.to_ppm(), file=f)


In [ ]:
# get all motifs and related factors from motif database
    motifs = read_motifs(database)
    motifsandfactors = {
        motif.id: [val for sublist in motif.factors.values() for val in sublist]
        for motif in motifs
    }

    # process the references
    logger.info("Now writing your motif2factors files.")
    # make sure the output dir exists
    pathlib.Path(f"{outdir}").mkdir(parents=True, exist_ok=True)
    for reference in new_reference:
        genome = genomepy.Genome(reference, genomes_dir, rebuild=False).name
        make_motif2factors(
            f"{outdir}/{genome}.{database}",
            new_reference=genome,
            database_references=database_references,
            motifsandfactors=motifsandfactors,
            database=orthogroup_db,
            strategy=strategy,
            motifs=motifs,
        )

## get the correct dataframes & columns

In [2]:
hg38 = genomepy.Annotation("/bank/genomes/GRCh38.p13")  # from Ensembl
hg38_genes = hg38.genes("gtf")
hg38_genes[0:3]

['AL391988.1', 'AC105411.1', 'AL590440.1']

In [3]:
a = genomepy.Annotation("raw/Smed.gtf", quiet=True)

t2g = a.gtf_dict("transcript_id", "gene_name")

a.genes("gtf")[0:3]

['SMEST009541001', 'SmMSTRG.15908', 'SmMSTRG.4940']

In [4]:
cur1 = pd.read_table("raw/planaria_tfs_Smed_curated_NewGenome.tsv")
cur1.head(2)

,gene,gene_NewAnnot_Guo,orthogroup,OG support,human orthologs,Support in ancestral node from human orthologs,Annotation in Guo New Genome,e value
0,Smed_SMEST022356001.1,SMEST022356001,AP-2.HG1.6:TFAP2A/TFAP2B/TFAP2C/TFAP2D/TFAP2E,75,TFAP2A/TFAP2B/TFAP2C/TFAP2D/TFAP2E,93.0/93.0/93.0/93.0/93.0,AFJ24713.1 ap2 [Schmidtea mediterranea],2.160000e-310
1,Smed_SMEST047168002.1,SMEST047168002,ARID_BRIGHT.HG1.0:ARID4A/ARID4B,100,ARID4A/ARID4B,95.0/95.0,XP_018644114.1 hypothetical protein Smp_124780...,5.380000e-44


In [5]:
# sanity check: using the correct column?
cur_genes = set(cur1["gene_NewAnnot_Guo"])
# cur_genes = set(cur1["gene"].str.replace("Smed_", "").str.replace("\.\d*", "", regex=True))
ann_genes = set(a.genes("gtf"))

print("overlap:", int(100* len(cur_genes & ann_genes) / min(len(cur_genes), len(ann_genes))), "%")

overlap: 99 %


In [6]:
cur2 = pd.read_table("raw/media-1_extraGuo.tsv")
cur2.head(2)

,Rink,Neiro,NewGuoGenome,Symbol,Old symbol,Description,TF group,TF class,Identification,RNAi,In situ,Reference,X1,X2,Xins,X1 percent,X2 percent,Xins percent,Jaspar MatrixID,Jaspar Evalue
0,SMESG000003328.1,MSTRG.707,SmMSTRG.707,fos-1,NaN,Fos proto-oncogene,FOS,Basic domain,1.0,Cyclopic blastemas and asymmetric tails,Ubiquitous,"(Wenemoser et al., 2012; Zhu et al., 2015)",73.531050,56.195082,82.571524,0.346358,0.264699,0.388942,0,0.0
1,SMESG000062649.1,MSTRG.18685,SmMSTRG.18685,batf-3,NaN,Basic leucine zipper transcription factor ATF-...,BATF,Basic domain,0.0,0,0,NaN,67.449376,56.338831,120.661601,0.275923,0.230472,0.493605,0,0.0


In [7]:
# sanity check: using the correct column?
cur_genes = set(cur2["NewGuoGenome"])
# cur_genes = set(cur2["Rink"].str.replace("\.\d*", "", regex=True))
ann_genes = set(a.genes("gtf"))

print("overlap:", int(100* len(cur_genes & ann_genes) / min(len(cur_genes), len(ann_genes))), "%")

overlap: 99 %


In [8]:
orthofinder = pd.read_table(
    "raw/Orthogroups.tsv",  # from the orthofinder output, see assembly_analyses.ipynb
    usecols=["Smed.pep", "GRCh38.p13.pep"]
).dropna()
orthofinder.columns = [ "GRCh38.p13", "Smed"]

for col in orthofinder.columns:
    orthofinder[col] = orthofinder[col].str.split("\||, ", regex=True).apply(lambda x: [g for g in x if g != "."])

orthofinder["GRCh38.p13"] = orthofinder["GRCh38.p13"].apply(lambda x: [g for g in x if g in hg38_genes])  # keep only gene names
orthofinder["Smed"] = orthofinder["Smed"].apply(lambda x: [t2g[t] for t in x if t in t2g])  # transcripts to gene names

ofs_genes = set([item for sublist in orthofinder["Smed"].to_list() for item in sublist])
print(f"{len(ofs_genes)} unique Smed genes in orthofinder output")
ofh_genes = set([item for sublist in orthofinder["GRCh38.p13"].to_list() for item in sublist])
print(f"{len(ofh_genes)} unique human genes in orthofinder output")
print("overlap:", int(100* len(ofs_genes & ann_genes) / min(len(ofs_genes), len(ann_genes))), "%")

orthofinder.head(3)

10077 unique Smed genes in orthofinder output
10585 unique human genes in orthofinder output
overlap: 100 %


,GRCh38.p13,Smed
0,"[AL583836.1, CYP17A1, CYP1A1, CYP1A2, CYP1B1, ...","[SMEST012534002, SMEST013700001, SMEST01370100..."
1,"[AC011473.4, AC109583.1, ACR, CELA1, CELA2A, C...","[SMEST002831001, SMEST025034001, SMEST04090700..."
2,"[AC004080.3, AC012531.3, BSX, EMX1, EMX2, GSX1...","[SmMSTRG.764, SMEST017219001, SMEST019482001, ..."


## get the orthologs from the curated table

In [9]:
orthologs = pd.DataFrame({"gene_name": a.genes("gtf"), "orthologs": pd.NA, "source": pd.NA})
orthologs.set_index("gene_name", inplace=True)
orthologs.sort_index(inplace=True)

print(len(orthologs), "genes in GTF")
orthologs.head(3)

29082 genes in GTF


,orthologs,source
gene_name,,
MT-1,<NA>,<NA>
SMEST000017001,<NA>,<NA>
SMEST000022001,<NA>,<NA>


In [10]:
# add manually curated orthologs

df = cur1[["gene_NewAnnot_Guo", "human orthologs"]].dropna().drop_duplicates()
df.columns = ["gene_name", "orthologs"]
df.set_index("gene_name", inplace=True)
df["source"] = "curated_human_orthologs"

# split orthologs
df["orthologs"] = df["orthologs"].str.split("/")
# # filter for presence in human gene list
# df["orthologs"] = df["orthologs"].apply(lambda orthologs: [g.upper() for g in orthologs if g.upper() in hg38_genes])
# # drop orthologs without any matching human factors
# df = df[df["orthologs"].map(lambda d: len(d)) > 0]

# missing = set(df[~df.index.isin(orthologs.index)].index)
# # remove missing
# df = df.loc[~df.index.isin(missing)]
# print(f"missing from gtf ({len(missing)}): {missing}")
# print()

# # which orthologs are missing from the orthologs df?
# len_df = len(df)
# df = df[orthologs.loc[df.index]["orthologs"].isna()]
# print(f"{len(df)} new orthologs added to the table. {len_df-len(df)} orthologs were already present")

# add these to the orthologs df
orthologs.loc[df.index] = df
print(f"table now contains {len(orthologs.dropna())} orthologs")

table now contains 255 orthologs


In [11]:
# add matching orthogroups

df = cur1[["gene_NewAnnot_Guo", "orthogroup"]].dropna().drop_duplicates()
df.columns = ["gene_name", "orthologs"]
df.set_index("gene_name", inplace=True)
df["source"] = "curated_orthogroups"

# split orthologs and filter for presence in human gene list
df["orthologs"] = df["orthologs"].str.split(":|;|/", regex=True)
df["orthologs"] = df["orthologs"].apply(lambda orthologs: [g.upper() for g in orthologs if g.upper() in hg38_genes])
# drop orthologs without any matching human factors
df = df[df["orthologs"].map(lambda d: len(d)) > 0]

missing = set(df[~df.index.isin(orthologs.index)].index)
# remove missing
df = df.loc[~df.index.isin(missing)]
print(f"missing from gtf ({len(missing)}): {missing}")
print()

# which orthologs are missing from the orthologs df?
len_df = len(df)
df = df[orthologs.loc[df.index]["orthologs"].isna()]
print(f"{len(df)} new orthologs added to the table. {len_df-len(df)} orthologs were already present")

# add these to the orthologs df
orthologs.loc[df.index] = df
print(f"table now contains {len(orthologs.dropna())} orthologs")

missing from gtf (0): set()

269 new orthologs added to the table. 255 orthologs were already present
table now contains 524 orthologs


# get the orthologs from Neiro et al.

In [12]:
# add symbol

df = cur2[["NewGuoGenome", "Symbol"]].dropna().drop_duplicates()
df.columns = ["gene_name", "orthologs"]
df.set_index("gene_name", inplace=True)
df["source"] = "neiro_symbol"

df["orthologs"] = df["orthologs"].str.upper().apply(lambda s : s if s in hg38_genes else (s.split("-")[0] if s.split("-")[0] in hg38_genes else pd.NA))
df = df[["orthologs", "source"]].dropna()

# split orthologs and filter for presence in human gene list
df["orthologs"] = df["orthologs"].str.split("/").apply(lambda orthologs: [g.upper() for g in orthologs if g.upper() in hg38_genes])
# drop orthologs without any matching human factors
df = df[df["orthologs"].map(lambda d: len(d)) > 0]

missing = set(df[~df.index.isin(orthologs.index)].index)
# remove missing
df = df.loc[~df.index.isin(missing)]
print(f"missing from gtf ({len(missing)}): {missing}")
print()

# which orthologs are missing from the orthologs df?
len_df = len(df)
df = df[orthologs.loc[df.index]["orthologs"].isna()]
print(f"{len(df)} new orthologs added to the table. {len_df-len(df)} orthologs were already present")

# add these to the orthologs df
orthologs.loc[df.index] = df
print(f"table now contains {len(orthologs.dropna())} orthologs")

missing from gtf (0): set()

102 new orthologs added to the table. 156 orthologs were already present
table now contains 626 orthologs


In [13]:
# add old symbol

df = cur2[["NewGuoGenome", "Old symbol"]].dropna().drop_duplicates()
df.columns = ["gene_name", "orthologs"]
df.set_index("gene_name", inplace=True)
df["source"] = "neiro_old_symbol"
df["orthologs"] = df["orthologs"].str.upper()

# split orthologs and filter for presence in human gene list
df["orthologs"] = df["orthologs"].str.split("/", regex=True).apply(lambda orthologs: [g.upper() for g in orthologs if g.upper() in hg38_genes])
# drop orthologs without any matching human factors
df = df[df["orthologs"].map(lambda d: len(d)) > 0]

missing = set(df[~df.index.isin(orthologs.index)].index)
# remove missing
df = df.loc[~df.index.isin(missing)]
print(f"missing from gtf ({len(missing)}): {missing}")
print()

# which orthologs are missing from the orthologs df?
len_df = len(df)
df = df[orthologs.loc[df.index]["orthologs"].isna()]
print(f"{len(df)} new orthologs added to the table. {len_df-len(df)} orthologs were already present")

# add these to the orthologs df
orthologs.loc[df.index] = df
print(f"table now contains {len(orthologs.dropna())} orthologs")

missing from gtf (0): set()

2 new orthologs added to the table. 25 orthologs were already present
table now contains 628 orthologs


# Optional: add orthofinder orthologs

In [14]:
missing_genes = orthologs.loc[orthologs.isna().any(1)].index.to_list()
print(missing_genes[:5])

df = orthofinder.copy()
df["smed_string"] = df["Smed"].apply(lambda gene_list: "/".join(gene_list))

max_o = 0
max_g = None
dist = []
to_add = []
for missing_gene in missing_genes:
    hits = df[df["smed_string"].str.contains(missing_gene)]["GRCh38.p13"]
    if len(hits) != 0:
        ortholog_genes = list(set([item for sublist in hits.to_list() for item in sublist]))
        l = len(ortholog_genes)
        if l > 1:
            dist.append(l)
        if l > max_o:
            max_o = l
            max_g = missing_gene
        to_add.append(
            [missing_gene, ortholog_genes, "orthofinder"]
        )


['MT-1', 'SMEST000017001', 'SMEST000022001', 'SMEST000026001', 'SMEST000031001']


In [15]:
# x_min = 20
# _ = plt.hist([n for n in dist if n >= x_min], bins=500)#, range=(6, xmax))# min(50, xmax))#, range=(6, xmax))
# print("distribution of human orthologs per Smed gene (outliers not shown)")
# print(f"y: Smed genes ({len(dist)} total), x: human orthologs (max {max(dist)})")

In [16]:
print(f"{len(to_add)} Smed genes (from the missing genes list) matched to orthologs")
print()

for n in range(1, 10):
    l = len([l for l in to_add if len(l[1]) == n])
    print(f"{l} Smed genes with {n} orthologs ({int(100*l/len(to_add))}%)")

# rest
l = len([l for l in to_add if len(l[1])> n])
print(f"{l} Smed genes with >{n} orthologs ({int(100*l/len(to_add))}%)")

10453 total Smed genes matched to orthologs (from the missing genes list)

4450 Smed genes with 1 orthologs (42%)
2093 Smed genes with 2 orthologs (20%)
1127 Smed genes with 3 orthologs (10%)
634 Smed genes with 4 orthologs (6%)
387 Smed genes with 5 orthologs (3%)
251 Smed genes with 6 orthologs (2%)
257 Smed genes with 7 orthologs (2%)
152 Smed genes with 8 orthologs (1%)
125 Smed genes with 9 orthologs (1%)
977 Smed genes with >9 orthologs (9%)


In [17]:
# some Smed genes have way too many potential orthologs to be useful

print(f"{len(to_add)} total genes (any number of matched human orthologs)")
max_orthologs = 4
print(f"{len([l for l in to_add if len(l[1]) <= max_orthologs])} genes (max {max_orthologs} matched human orthologs)")

# example
[l for l in to_add[0:3] if len(l[1]) <= max_orthologs]

10453 total genes (any number of matched human orthologs)
8304 genes (max 4 matched human orthologs)


[['SMEST000041001', ['GFM2'], 'orthofinder'],
 ['SMEST000042001', ['AC093525.1', 'AMDHD2'], 'orthofinder'],
 ['SMEST000046001', ['GFM2'], 'orthofinder']]

In [18]:
df = pd.DataFrame([l for l in to_add if len(l[1]) <= max_orthologs], columns=["gene_name", "orthologs", "source"]).set_index("gene_name")
df.head(3)

,orthologs,source
gene_name,,
SMEST000041001,[GFM2],orthofinder
SMEST000042001,"[AC093525.1, AMDHD2]",orthofinder
SMEST000046001,[GFM2],orthofinder


In [19]:
assert len(set(df.index) & set(missing_genes)) / len(set(df.index)) == 1
assert len(set(df.index) & set(orthologs.dropna().index)) == 0

# add these to the orthologs df
orthologs.loc[df.index] = df
print(f"{len(df)} new orthologs added to the table.")
print(f"table now contains {len(orthologs.dropna())} orthologs")

8304 new orthologs added to the table.
table now contains 8932 orthologs


# save

In [20]:
print(f"{len(set([item for sublist in orthologs.orthologs.dropna().to_list() for item in sublist]))} total human TFs in orthologs")
print()

missing = orthologs[orthologs.isna().any(1)].index.unique().to_list()
print(f"{len(orthologs.dropna())} Smed genes with orthologs")
print(f"{len(missing)} Smed genes without orthologs")

8325 total human TFs in orthologs

8932 Smed genes with orthologs
20150 Smed genes without orthologs


In [21]:
orthologs = orthologs.dropna()
orthologs["orthologs"] = orthologs["orthologs"].apply(lambda x: "/".join(x))

orthologs.head(5)

,orthologs,source
gene_name,,
SMEST000041001,GFM2,orthofinder
SMEST000042001,AC093525.1/AMDHD2,orthofinder
SMEST000046001,GFM2,orthofinder
SMEST000047001,GFM2,orthofinder
SMEST000092001,CHD6/CHD8/CHD9/CHD7,orthofinder


In [23]:
# sanity check (that no overwrites happened): orthologs per source
orthologs.groupby("source").size()

source
curated_human_orthologs     255
curated_orthogroups         269
neiro_old_symbol              2
neiro_symbol                102
orthofinder                8304
dtype: int64

In [22]:
orthologs.to_csv("orthologs_with_orthofinder.tsv", sep="\t")